# Onboarding new models

Notebook walking through steps to implement new features in the model module

Created by: Jacob A Rose  
Created On: Tuesday, June 22nd, 2021

<!-- Previous description:  
(Notebook walking through some best practice unit tests for performing sequentially in order to debug a misbehaving black box model.) -->

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "7"

from lightning_hydra_classifiers.models import base, heads, backbones
from lightning_hydra_classifiers.models.heads.classifier import Classifier
from contrastive_learning.data.pytorch.datamodules import get_datamodule
import torch
import pytorch_lightning as pl

from torchinfo import summary
from rich import print as pp
from torch import nn
from typing import Tuple

from omegaconf import OmegaConf


from lightning_hydra_classifiers.utils.train_basic_utils import configure_trainer

/media/data/conda/jrose3/envs/sequoia/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


In [2]:
from torchinfo import summary
def log_model_summary(model: nn.Module,
                      working_dir: str,
                      input_size: Tuple[int],
                      full_summary: bool=True,
                      verbose: bool=1):
    """
    produce a text file with the model summary
    
    TODO: Add this to Eval Plugins
    
    log_model_summary(model=model,
                  working_dir=working_dir,
                  input_size=(1, data_config.channels, *data_config.image_size),
                  full_summary=True)

    """

    if full_summary:
        col_names=("kernel_size", "input_size","output_size", "num_params", "mult_adds")
    else:
        col_names=("input_size","output_size", "num_params")

    model_summary = summary(model.cuda(),
                            input_size=input_size,
                            row_settings=('depth', 'var_names'),
                            col_names=col_names,
                            verbose=verbose)

    model_summary_file = os.path.join(working_dir, 'model', f'{MODEL_NAME}_model_summary.txt')
    os.makedirs(os.path.dirname(model_summary_file), exist_ok=True)

    with open(model_summary_file, "w") as f:
        f.write(str(model_summary))
        
    return model_summary

def get_configs(DATASET_NAME: str="PNAS_family_100_512",
                MODEL_NAME: str="resnet18"):

    data_config = OmegaConf.create(
                    dict(
                         name=DATASET_NAME,
                         batch_size=12,
                         val_split=0.2,
                         num_workers=0,
                         seed=None,
                         debug=False,
                         normalize=True,
                         image_size=(512,512), #(224,224),
                         channels=3,
                         dataset_dir=None
                        )
    )

    model_config = OmegaConf.create(
                    dict(
                         name=MODEL_NAME,
                         pretrained=True,
                         input_size=(2, data_config.channels, *data_config.image_size)
                        )
    )
    
    
    trainer_config = OmegaConf.create(
                    dict(    
#                          _target_ = pl.Trainer,
                         gpus = 1,
                          min_epochs = 1,
                         max_epochs = 40,
                         weights_summary = "top",
                         progress_bar_refresh_rate = 10,
                         profiler = "simple",
                         log_every_n_steps = 50,
                         fast_dev_run = False,
                         limit_train_batches = 1.0,
                         limit_val_batches = 1.0,
                         auto_lr_find = False,
                         auto_scale_batch_size = False
                        )
    )

    


    return data_config, model_config, trainer_config

def setup_train(data_config,
                model_config,
                trainer_config,
                working_dir: str,
                verbose: bool=1):

    datamodule = get_datamodule(data_config = data_config)
    model_config.num_classes = len(datamodule.classes)

#     backbone = backbones.build_model(model_config.name,
#                                      pretrained=model_config.pretrained)

    model = Classifier(backbone_name=model_config.name,
                       num_classes=model_config.num_classes,
                       finetune=True)
    
    finetuning_callback = MilestonesFinetuning(**self.config_init['finetuning'])
    callbacks = [finetuning_callback]
    
    train_config = OmegaConf.to_container(trainer_config, resolve=True)
    trainer = pl.Trainer(**train_config,
                         callbacks=callbacks)
    

    log_model_summary(model=model,
                      working_dir=working_dir,
                      input_size=list(model_config.input_size),
                      full_summary=True,
                      verbose=verbose)

    return datamodule, model, trainer


In [8]:
criterion = nn.CrossEntropyLoss()

criterion

In [3]:
# working_dir = "/media/data/jacob/GitHub/lightning-hydra-classifiers/notebooks/playground_results"
# os.makedirs(working_dir, exist_ok=True)

# MODEL_NAMES = ['resnet101']
# MODEL_NAMES = ['resnet18', 'resnet34', 'resnet50', 'resnet101',
#                'resnet152', 'resnext50_32x4d', 'resnext101_32x8d']
# DATASET_NAME="PNAS_family_100_512"
# for MODEL_NAME in MODEL_NAMES:
#     print(f'Summarizing model {MODEL_NAME}')

#     data_config, model_config, trainer_config = get_configs(DATASET_NAME=DATASET_NAME,
#                                                             MODEL_NAME=MODEL_NAME)

#     datamodule, model = get_data_and_model(data_config,
#                                            model_config,
#                                            working_dir=working_dir,
#                                            verbose=0)

## Begin training

In [4]:
working_dir = "/media/data/jacob/GitHub/lightning-hydra-classifiers/notebooks/playground_results"
os.makedirs(working_dir, exist_ok=True)

MODEL_NAME = 'resnet101'
DATASET_NAME="PNAS_family_100_512"


data_config, model_config, trainer_config = get_configs(DATASET_NAME=DATASET_NAME,
                                                        MODEL_NAME=MODEL_NAME)


datamodule, model, trainer = setup_train(data_config,
                                         model_config,
                                         trainer_config,
                                         working_dir=working_dir,
                                         verbose=0)

# trainer.fit(model, datamodule=datamodule)

dir(model)

Warning, no values for name or dataset_dir provided to constructor.
Warning, no values for name or dataset_dir provided to constructor.
Warning, no values for name or dataset_dir provided to constructor.
Warning, no values for name or dataset_dir provided to constructor.
Warning, no values for name or dataset_dir provided to constructor.
Warning, no values for name or dataset_dir provided to constructor.
Warning, no values for name or dataset_dir provided to constructor.
Warning, no values for name or dataset_dir provided to constructor.


GPU available: True, used: True
TPU available: False, using: 0 TPU cores


['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_features_dim',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'add_module',
 'apply',
 'backbone',
 'bfloat16',
 'bottleneck',
 'buffers',
 'children',
 'cpu'

In [6]:
dict(model.named_children()).values()

dict_values([ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1)

## Model Summary

In [4]:
x, y = next(iter(datamodule.train_dataloader()))
y_hat = model(x[:16,...].cuda())


pp(y_hat[0].shape, y_hat[1].shape)

torch.Size([16, 19])
torch.Size([16, 2048])

In [10]:
# backbone_summary_file = os.path.join(working_dir, 'model', f'{MODEL_NAME}_backbone_summary.txt')
# backbone_summary = summary(backbone.cuda(),
#                            input_size=(1, data_config.channels, *data_config.image_size),
#                            row_settings=('depth', 'var_names'),
#                            col_names=("kernel_size", "input_size","output_size", "num_params", "mult_adds"),
#                            verbose=1)
# # print(backbone_summary)

# with open(backbone_summary_file, "w") as f:
#     f.write(str(backbone_summary))

# heads.classifier.Classifier

# dir(backbones.resnet)